# Download Weather NOAA Data

The Global Historical Climatology Network daily (GHCNd) is an integrated database of daily climate summaries from land surface stations across the globe. GHCNd is made up of daily climate records from numerous sources that have been integrated and subjected to a common suite of quality assurance reviews

AWS CLI Access (No AWS account required)
``` bash
aws s3 ls --no-sign-request s3://noaa-ghcn-pds/
```

In [7]:
import io
from pathlib import Path

import boto3
import pandas as pd
from botocore import UNSIGNED
from botocore.client import Config

In [12]:
DATA_PATH = Path().resolve().parent.parent/'data'/'noaa'
print(DATA_PATH)

/home/pyuser/workspace/data/noaa


In [14]:
BUCKET = 'noaa-ghcn-pds'

## Download a file

In [5]:
# create a s3 client without signin
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [13]:
# Download the file from S3
s3_client.download_file(BUCKET, 'ghcnd-stations.txt', DATA_PATH/'ghcnd-stations.txt')

### Download and read directly

|Variable|Columns|Type|Example|
|-----|-----|-----|-----|
|ID|1-11|Character|EI000003980|
|LATITUDE|13-20|Real|55.3717|
|LONGITUDE|22-30|Real|-7.3400|
|ELEVATION|32-37|Real|21.0|
|STATE|39-40|Character|
|NAME|42-71|Character|MALIN HEAD|
|GSN FLAG|73-75|Character|GSN|
|HCN/CRN FLAG|77-79|Character|
|WMO ID|81-85|Character|03980|

In [26]:
obj = s3_client.get_object(Bucket=BUCKET, Key='ghcnd-stations.txt')
data = io.BytesIO(obj['Body'].read())
pd.read_fwf(
    data,
    colspecs=[
        (1, 12),
        (13, 21),
        (22, 31),
        (32, 38),
        (39, 41),
        (42, 72),
        (73, 76),
        (77, 80),
        (81, 86),
    ],
    header=None,
    names=[
        'ID',
        'LATITUDE',
        'LONGITUDE',
        'ELEVATION',
        'STATE',
        'NAME',
        'GSN FLAG',
        'HCN/CRN FLAG',
        'WMO ID',
    ]
).head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,CW00011604,17.1167,-61.7833,10.1,NaN,T JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,CW00011647,17.1333,-61.7833,19.2,NaN,T JOHNS,NaN,NaN,NaN
2,E000041196,25.3330,55.5170,34.0,NaN,HARJAH INTER. AIRP,SN,NaN,1196.0
3,EM00041194,25.2550,55.3640,10.4,NaN,UBAI INTL,NaN,NaN,1194.0
4,EM00041217,24.4330,54.6510,26.8,NaN,BU DHABI INTL,NaN,NaN,1217.0


## References

- [docs.opendata.aws/noaa-ghcn-pds](https://docs.opendata.aws/noaa-ghcn-pds/readme.html)
- [AWS S3 Explorer](https://noaa-ghcn-pds.s3.amazonaws.com/index.html)
- [Registry of Open Data on AWS](https://registry.opendata.aws/noaa-ghcn/)
- [github.com/awslabs/open-data-docs](https://github.com/awslabs/open-data-docs)